In [1]:
# for classical solver (simulated annealing)
import dimod
import operator
import dwavebinarycsp
import numpy as np
import math

In [3]:
rows = 'ABCDEFGHI'
cols = '123456789'
boxes = [[("{}{}".format(r, c)) for c in cols] for r in rows]  # declare variables for each box in the puzzle
square_units = [ [ x+y for x in A for y in B ] for A in ('ABC','DEF','GHI') for B in ('123','456','789') ]

In [4]:
def print_board(board):
    if isinstance(board, str):     
        board = string_to_tuple(board)
    for row, _boxes in enumerate(boxes):
        if row and row % 3 == 0:
            print('-'*9+"|"+'-'*9+"|"+'-'*9)
        for col, box in enumerate(_boxes):
            if col and col % 3 == 0:
                print('|', end='')
            print(' {} '.format((int(board[row][col]) or '-')), end='')
        print()
    print()

In [5]:
board = ((0, 0, 3, 0, 2, 0, 6, 0, 0),
         (9, 0, 0, 3, 0, 5, 0, 0, 1),
         (0, 0, 1, 8, 0, 6, 4, 0, 0),
         (0, 0, 8, 1, 0, 2, 9, 0, 0),
         (7, 0, 0, 0, 0, 0, 0, 0, 8),
         (0, 0, 6, 7, 0, 8, 2, 0, 0),
         (0, 0, 2, 6, 0, 9, 5, 0, 0),
         (8, 0, 0, 2, 0, 3, 0, 0, 9),
         (0, 0, 5, 0, 1, 0, 3, 0, 0))
print_board(board)

 -  -  3 | -  2  - | 6  -  - 
 9  -  - | 3  -  5 | -  -  1 
 -  -  1 | 8  -  6 | 4  -  - 
---------|---------|---------
 -  -  8 | 1  -  2 | 9  -  - 
 7  -  - | -  -  - | -  -  8 
 -  -  6 | 7  -  8 | 2  -  - 
---------|---------|---------
 -  -  2 | 6  -  9 | 5  -  - 
 8  -  - | 2  -  3 | -  -  9 
 -  -  5 | -  1  - | 3  -  - 



In [6]:
board = ((0, 0, 0, 8, 9, 0, 0, 0, 0),
         (2, 6, 0, 0, 0, 4, 0, 0, 0),
         (0, 0, 4, 0, 0, 0, 0, 0, 0),
         (0, 0, 5, 3, 0, 0, 4, 0, 9),
         (0, 8, 0, 0, 0, 0, 7, 5, 0),
         (6, 0, 0, 0, 0, 0, 0, 0, 3),
         (0, 7, 0, 0, 2, 6, 0, 0, 4),
         (1, 2, 0, 5, 0, 3, 0, 0, 0),
         (0, 0, 6, 0, 0, 0, 2, 0, 0))
print_board(board)

 -  -  - | 8  9  - | -  -  - 
 2  6  - | -  -  4 | -  -  - 
 -  -  4 | -  -  - | -  -  - 
---------|---------|---------
 -  -  5 | 3  -  - | 4  -  9 
 -  8  - | -  -  - | 7  5  - 
 6  -  - | -  -  - | -  -  3 
---------|---------|---------
 -  7  - | -  2  6 | -  -  4 
 1  2  - | 5  -  3 | -  -  - 
 -  -  6 | -  -  - | 2  -  - 



In [6]:
correct_board = ((5, 3, 7, 8, 9, 1, 6, 4, 2),
         (2, 6, 1, 7, 5, 4, 3, 9, 8),
         (8, 9, 4, 6, 3, 2, 1, 7, 5),
         (7, 1, 5, 3, 6, 8, 4, 2, 9),
         (3, 8, 2, 4, 1, 9, 7, 5, 6),
         (6, 4, 9, 2, 7, 5, 8, 1, 3),
         (9, 7, 3, 1, 2, 6, 5, 8, 4),
         (1, 2, 8, 5, 4, 3, 9, 6, 7),
         (0, 5, 6, 9, 8, 7, 2, 3, 1))
print_board(correct_board)

 5  3  7 | 8  9  1 | 6  4  2 
 2  6  1 | 7  5  4 | 3  9  8 
 8  9  4 | 6  3  2 | 1  7  5 
---------|---------|---------
 7  1  5 | 3  6  8 | 4  2  9 
 3  8  2 | 4  1  9 | 7  5  6 
 6  4  9 | 2  7  5 | 8  1  3 
---------|---------|---------
 9  7  3 | 1  2  6 | 5  8  4 
 1  2  8 | 5  4  3 | 9  6  7 
 -  5  6 | 9  8  7 | 2  3  1 



In [7]:
from itertools import product

def check_sudoku(grid):
    """Validate a sudoku solution.

    Given a grid as a list of lists, return None if it is ill-formed,
    False if it is invalid, or True if it is a valid solution.
    """
    error_count=0
    # Check that the grid is 9x9.
    if len(grid) != 9 or not all(len(row) == 9 for row in grid):
        error_count+=1
    DIGITS = set(range(1, 10))

    # Check that each number appears exactly once per row
    if not all(set(row) == DIGITS for row in grid):
        error_count+=1

    # Check that each number appears exactly once per column
    columns = [[row[c] for row in grid] for c in range(9)]
    if not all(set(col) == DIGITS for col in columns):
        error_count+=1

    # Check that each number appears exactly once per 3x3 grid
    THREES = [(0, 1, 2), (3, 4, 5), (6, 7, 8)]
    for row_block, col_block in product(THREES, THREES):
        block = [grid[r][c] for r, c in product(row_block, col_block)]
        if set(block) != DIGITS:
            error_count+=1


    return error_count


In [8]:
check_sudoku(board)

11

In [9]:
check_sudoku(correct_board)

3

In [10]:
def encode_board_to_binary(board):
    binary_board= np.zeros((9, 9, 9)).tolist()
    for row_index, row in enumerate(board):
        for column_index, cell in enumerate(row):
            if cell>0:
                binary_board[cell-1][row_index][column_index]=1    
    return binary_board 

def decode_board_from_binary(binary_board):
    board= np.zeros((9, 9))
    for k, color in enumerate(binary_board):
        for i, row in enumerate(color):
            for j, cell_value in enumerate(row):
                if cell_value>0:
                    board[i][j]+=int(k+1)
                else:
                    board[i][j]+=int(0)
    return board 

In [11]:
# for exmaple: relabel binary 
# variable x_i_j_k-> x_1_1_1 should equal x_1 x_1_1_2 = x_2 and so forth..
def encode_var_labels(i,j,k,N=9):
    #return (i-1)*N*N+(j-1)*N+k
    counter=0
    for a in range(i):
        for b in range(j):
            for c in range(k):
                pass
    return (a*N*N)+(b*N)+c+1 
    
def decode_var_labels(index, N=9):
    for i in range (N):
        for j in range(N):
            for k in range(N):
                if (i*N*N)+(j*N)+k+1 == index:
                    return[i+1,j+1,k+1]

In [12]:
constant = 0
N = 9
cell_qubo = {}
linear = {}
quad = {}
penalty_weight = 1
for i in range(1,N+1):
        for j in range(1,N+1):
            for k1 in range(1,N+1):  
                    var_1 = encode_var_labels(i,j,k1)
                    for k2 in range(1,N+1):
                            var_2 = encode_var_labels(i,j,k2)
                            if var_1 == var_2:
                                linear[var_1] = -1*penalty_weight
                            else:
                                quad[var_1,var_2]= 2*penalty_weight
                        #linear[var_1] = 2
                    
            constant+=1
            

cell_qubo[()] = constant*penalty_weight
cell_qubo["linear"] = linear
cell_qubo["quadratic"] = quad

#bqm = dimod.BinaryQuadraticModel(cell_qubo,constant,dimod.Vartype.BINARY)

In [13]:
penalty_weight=1
constant = 0
N = 9
column_qubo = {}
lin_column={}
quad_column={}
binary_board = encode_board_to_binary(board)
for k in range(1,N):
        for j in range(1,N):
            for i1 in range(1,N):
                var_1 = encode_var_labels(i1,j,k)
                for i2 in range(1,N):               
                    var_2 = encode_var_labels(i2,j,k)
                    if var_1 == var_2:
                        lin_column[var_1] = -1*penalty_weight
                    else:
                        quad_column[var_1,var_2] = 2*penalty_weight
        constant+=1
        
column_qubo[()] = constant*penalty_weight
column_qubo['linear'] = lin_column
column_qubo['quadratic'] = quad_column

In [14]:
constant = 0
N = 9
row_qubo = {}
linear_row ={}
quadratic_row={}
penalty_weight=2
binary_board = encode_board_to_binary(board)
for k in range(1,N):
        for i in range(1,N):
            for j1 in range(1,N):
                var_1 = encode_var_labels(i,j1,k)
                for j2 in range(1,N):      
                    var_2 = encode_var_labels(i,j2,k)
                    if var_1 == var_2:
                        linear_row[var_1] = -1*penalty_weight
                    else:
                        quadratic_row[var_1, var_2] = 2*penalty_weight
        constant+=1
        
row_qubo[()] = constant*penalty_weight
row_qubo['linear'] = linear_row
row_qubo['quadratic'] = quadratic_row

In [15]:
binary_board = encode_board_to_binary(board)
hint = []
penalty_weight = 4

for k, color in enumerate(binary_board):
    for i, row in enumerate(color):
        for j, cell in enumerate(row):
            if cell>0:
                hint.append([i+1,j+1,k+1])
                
hint_qubo = {}
constant=0
for (i, j, k) in hint:
    re_label = encode_var_labels(i,j,k) 
    hint_qubo[re_label] = -1*penalty_weight
    constant += 1

hint_qubo[()] = constant*penalty_weight


In [16]:
from math import sqrt
constant = 0
N = 9
duplicate_qubo = {}
dupli_linear = {}
dupli_quadr = {}
sqrtN = int(sqrt(N))
for grid_i in range(sqrtN):
    for grid_j in range(sqrtN):
        for k in range(N):
            # there can be only one k in the same subgrid.
            for i1 in range(grid_i * 3, grid_i * 3 + 3):
                for j1 in range(grid_j * 3, grid_j * 3 + 3):
                    for i2 in range(grid_i * 3, grid_i * 3 + 3):
                        var_1 = encode_var_labels(i1+1,j1+1,k+1)
                        for j2 in range(grid_j * 3, grid_j * 3 + 3):
                            var_2 = encode_var_labels(i2+1,j2+1,k+1)
                            if var_1 == var_2:
                                dupli_linear[var_1] = -1
                            else:
                                dupli_quadr[var_1,var_2] = 1
            constant+=1

            
duplicate_qubo[()] = constant
duplicate_qubo['linear'] = dupli_linear
duplicate_qubo['quadratic'] = dupli_quadr

In [17]:
result_qubo_linear = {}
result_qubo_quadratic = {}

In [18]:
for index, value in hint_qubo.items():
    if result_qubo_linear.get(index):
        result_qubo_linear[index] += value
        
    else:
        result_qubo_linear[index] = value

In [19]:
for index, value in cell_qubo['linear'].items():
    if result_qubo_linear.get(index):
        result_qubo_linear[index] += value
        
    else:
        result_qubo_linear[index] = value

In [20]:
for index, value in cell_qubo['quadratic'].items():
    if result_qubo_quadratic.get(index):
        result_qubo_quadratic[index] += value
        
    else:
        result_qubo_quadratic[index] = value

In [21]:
for index, value in column_qubo['linear'].items():
    if result_qubo_linear.get(index):
        result_qubo_linear[index] += value
    else:
        result_qubo_linear[index] = value

In [22]:
for index, value in column_qubo['quadratic'].items():
    if result_qubo_quadratic.get(index):
        result_qubo_quadratic[index] += value
    else:
        result_qubo_quadratic[index] = value

In [23]:
for index, value in row_qubo['linear'].items():
    if result_qubo_linear.get(index):
        result_qubo_linear[index] += value
        
    else:
        result_qubo_linear[index] = value

In [24]:
for index, value in row_qubo['quadratic'].items():
    if result_qubo_quadratic.get(index):
        result_qubo_quadratic[index] += value
        
    else:
        result_qubo_quadratic[index] = value

In [25]:
for index, value in duplicate_qubo['linear'].items():
    if result_qubo_linear.get(index):
        result_qubo_linear[index] += value
        
    else:
        result_qubo_linear[index] = value

In [26]:
for index, value in duplicate_qubo['quadratic'].items():
    if result_qubo_quadratic.get(index):
        result_qubo_quadratic[index] += value
        
    else:
        result_qubo_quadratic[index] = value

In [27]:
constant/2

40.5

In [28]:
constant= cell_qubo.get((),0) + hint_qubo.get((),0) + row_qubo.get((),0) + column_qubo.get((),0) + duplicate_qubo.get((),0)
bqm = dimod.BinaryQuadraticModel(result_qubo_linear,result_qubo_quadratic, constant,dimod.Vartype.BINARY)

In [29]:
# test run
import neal
sampler = neal.SimulatedAnnealingSampler()
sample_set=sampler.sample(bqm, num_reads=30,num_sweeps=2222)


In [30]:
from random import randrange
import time

def find_optimal_solution(best_solution_global, current_solution, current_energy):
    best_solution = {}
    error_count=20
    iteration = 0
    sampler = neal.SimulatedAnnealingSampler()
    start_time = time.perf_counter()
    while error_count>0:
        """
        sample_set = sampler.sample(bqm, seed=1234, beta_range=[0.1, 4.2],
                                        num_reads=10, num_sweeps=20000,
                                       beta_schedule_type='geometric')
        """
        random_seed = randrange(9000)
        #random_seed = 5832
        num_reads = randrange(12000)
        num_sweeps = randrange(60000)
        num_reads = 5
        num_sweeps=50
        #num_sweeps*=iteration
        pre_anneal = time.perf_counter()
        sample_set=sampler.sample(bqm,seed=random_seed ,num_reads=num_reads,num_sweeps=num_sweeps)
        iteration+=1
        post_anneal = time.perf_counter()
        
        annealing_time = post_anneal-pre_anneal
        for solution, energy in sample_set.data(['sample', 'energy']):
            binary_solution_board= np.zeros((9, 9, 9))
            for index, value in solution.items():
                if type(index) is int and index>0:
                    board_index = decode_var_labels(index)
                    binary_solution_board[board_index[2]-1][board_index[0]-1][board_index[1]-1] = value
            solution_board=decode_board_from_binary(binary_solution_board)
            error_count_temp = check_sudoku(solution_board)
            
            
            
            current_solution.append(1)
            current_energy.append(energy)
            current_solution[:] = []
            current_energy[:] = []
            current_solution.append(solution)
            current_energy.append(energy)
            
            
        overall_time = time.perf_counter()-start_time
        if error_count_temp<error_count:
            best_solution = solution_board
            best_solution_global.append(solution)
            error_count=error_count_temp
            print("\nError Count:",error_count, "iteration:", iteration, "Energy:", energy,
                  "Seed:", random_seed, "num_reads:", num_reads, "sweeps:", num_sweeps, 
                 "Annealing Time:", annealing_time, "Overall Time:", overall_time)
            
        if iteration%10 == 0:
            print("\nCurrent State:",
                  "Error Count:",error_count_temp, "iteration:", iteration, "Energy:", energy,
                 "Seed:", random_seed, "num_reads:", num_reads, "sweeps:", num_sweeps,
                 "Annealing Time:", annealing_time, "Overall Time:", overall_time)
            

        

In [45]:
import multiprocessing
manager = multiprocessing.Manager()
best_solution = manager.list()
current_solution = manager.list()
current_energy = manager.list()


process = multiprocessing.Process(target=find_optimal_solution, 
            
                                  args= (best_solution, current_solution, current_energy))
process.start()




Error Count: 6 iteration: 1 Energy: -105.0 Seed: 6610 num_reads: 5 sweeps: 50 Annealing Time: 0.12621119400137104 Overall Time: 0.3090316919842735

Current State: Error Count: 6 iteration: 10 Energy: -101.0 Seed: 7591 num_reads: 5 sweeps: 50 Annealing Time: 0.12421651097247377 Overall Time: 3.237474827998085

Error Count: 5 iteration: 18 Energy: -94.0 Seed: 1933 num_reads: 5 sweeps: 50 Annealing Time: 0.1260571299935691 Overall Time: 5.816639102995396

Current State: Error Count: 9 iteration: 20 Energy: -107.0 Seed: 133 num_reads: 5 sweeps: 50 Annealing Time: 0.1443088899832219 Overall Time: 6.618094535981072

Current State: Error Count: 9 iteration: 30 Energy: -97.0 Seed: 6339 num_reads: 5 sweeps: 50 Annealing Time: 0.11300338001456112 Overall Time: 9.864797967980849

Current State: Error Count: 8 iteration: 40 Energy: -82.0 Seed: 4093 num_reads: 5 sweeps: 50 Annealing Time: 0.12409657999523915 Overall Time: 13.147613757988438

Error Count: 4 iteration: 43 Energy: -97.0 Seed: 4531 nu

In [46]:
import time
time.sleep(10)
#wait for terminating in order to have  
#results when doing a (quick) autorun

process.terminate()
process

<Process name='Process-4' pid=1534384 parent=1533466 started>

In [33]:
"""
from tabu import TabuSampler
solver = TabuSampler()
sample_set = solver.sample(bqm,num_reads=100)
"""

'\nfrom tabu import TabuSampler\nsolver = TabuSampler()\nsample_set = solver.sample(bqm,num_reads=100)\n'

In [47]:
#sample_set = list(current_solution)
sample_set = list(best_solution)

In [48]:
solution={}
solution_2={}
solution_3={}
for number, solution in enumerate(sample_set):
    if number==0:
        solution = solution
    elif number==1:
        solution_2 = solution
    elif number==2:
        solution_3 = solution
        
    
last_solution = solution

In [49]:
binary_solution_board= np.zeros((9, 9, 9))
for index, value in solution.items():
    if type(index) is int and index>0:
        board_index = decode_var_labels(index)
        binary_solution_board[board_index[2]-1][board_index[0]-1][board_index[1]-1] = value


In [50]:
solution_board=decode_board_from_binary(binary_solution_board)
solution_board

array([[5., 6., 3., 8., 2., 9., 1., 4., 3.],
       [2., 1., 7., 4., 3., 5., 6., 8., 7.],
       [4., 3., 8., 6., 1., 7., 5., 2., 9.],
       [7., 8., 1., 3., 5., 2., 4., 6., 9.],
       [3., 4., 9., 1., 6., 8., 7., 5., 2.],
       [2., 5., 6., 9., 7., 4., 8., 1., 3.],
       [8., 7., 4., 5., 9., 3., 2., 6., 4.],
       [1., 2., 5., 7., 4., 6., 9., 3., 7.],
       [9., 3., 6., 1., 8., 2., 2., 8., 1.]])

In [51]:
print_board(board)

 -  -  - | 8  9  - | -  -  - 
 2  6  - | -  -  4 | -  -  - 
 -  -  4 | -  -  - | -  -  - 
---------|---------|---------
 -  -  5 | 3  -  - | 4  -  9 
 -  8  - | -  -  - | 7  5  - 
 6  -  - | -  -  - | -  -  3 
---------|---------|---------
 -  7  - | -  2  6 | -  -  4 
 1  2  - | 5  -  3 | -  -  - 
 -  -  6 | -  -  - | 2  -  - 



In [52]:
print_board(solution_board)
print("Error Count:",check_sudoku(solution_board))
print("BQM Energy:", bqm.energy(solution))

 5  6  3 | 8  2  9 | 1  4  3 
 2  1  7 | 4  3  5 | 6  8  7 
 4  3  8 | 6  1  7 | 5  2  9 
---------|---------|---------
 7  8  1 | 3  5  2 | 4  6  9 
 3  4  9 | 1  6  8 | 7  5  2 
 2  5  6 | 9  7  4 | 8  1  3 
---------|---------|---------
 8  7  4 | 5  9  3 | 2  6  4 
 1  2  5 | 7  4  6 | 9  3  7 
 9  3  6 | 1  8  2 | 2  8  1 

Error Count: 4
BQM Energy: -97.0


In [53]:
binary_solution_board= np.zeros((9, 9, 9))
for index, value in solution_2.items():
    if type(index) is int and index>0:
        board_index = decode_var_labels(index)
        binary_solution_board[board_index[2]-1][board_index[0]-1][board_index[1]-1] = value
print_board(solution_board)
print("Error Count:",check_sudoku(solution_board))

 5  6  3 | 8  2  9 | 1  4  3 
 2  1  7 | 4  3  5 | 6  8  7 
 4  3  8 | 6  1  7 | 5  2  9 
---------|---------|---------
 7  8  1 | 3  5  2 | 4  6  9 
 3  4  9 | 1  6  8 | 7  5  2 
 2  5  6 | 9  7  4 | 8  1  3 
---------|---------|---------
 8  7  4 | 5  9  3 | 2  6  4 
 1  2  5 | 7  4  6 | 9  3  7 
 9  3  6 | 1  8  2 | 2  8  1 

Error Count: 4


In [54]:
binary_solution_board= np.zeros((9, 9, 9))
for index, value in solution_3.items():
    if type(index) is int and index>0:
        board_index = decode_var_labels(index)
        binary_solution_board[board_index[2]-1][board_index[0]-1][board_index[1]-1] = value
print_board(solution_board)
print("Error Count:",check_sudoku(solution_board))

 5  6  3 | 8  2  9 | 1  4  3 
 2  1  7 | 4  3  5 | 6  8  7 
 4  3  8 | 6  1  7 | 5  2  9 
---------|---------|---------
 7  8  1 | 3  5  2 | 4  6  9 
 3  4  9 | 1  6  8 | 7  5  2 
 2  5  6 | 9  7  4 | 8  1  3 
---------|---------|---------
 8  7  4 | 5  9  3 | 2  6  4 
 1  2  5 | 7  4  6 | 9  3  7 
 9  3  6 | 1  8  2 | 2  8  1 

Error Count: 4


In [56]:
binary_solution_board= np.zeros((9, 9, 9))
for index, value in last_solution.items():
    if type(index) is int and index>0:
        board_index = decode_var_labels(index)
        binary_solution_board[board_index[2]-1][board_index[0]-1][board_index[1]-1] = value
print_board(solution_board)
print("Error Count:",check_sudoku(solution_board))

 5  6  3 | 8  2  9 | 1  4  3 
 2  1  7 | 4  3  5 | 6  8  7 
 4  3  8 | 6  1  7 | 5  2  9 
---------|---------|---------
 7  8  1 | 3  5  2 | 4  6  9 
 3  4  9 | 1  6  8 | 7  5  2 
 2  5  6 | 9  7  4 | 8  1  3 
---------|---------|---------
 8  7  4 | 5  9  3 | 2  6  4 
 1  2  5 | 7  4  6 | 9  3  7 
 9  3  6 | 1  8  2 | 2  8  1 

Error Count: 4


In [57]:
bqm.energy(solution)

-97.0

In [58]:
binary_correct_board =  encode_board_to_binary(correct_board)
temp = []
for k, color in enumerate(binary_correct_board):
    for i, row in enumerate(color):
        for j, cell in enumerate(row):
            if cell>0:
                hint.append([i+1,j+1,k+1])
                
correct_solution_qubo = {}
constant=0
for (i, j, k) in hint:
    var = encode_var_labels(i,j,k) 
    correct_solution_qubo[var] = 1

for index in range(1,730):
    if correct_solution_qubo.get(index,0) == 0:
        correct_solution_qubo[index] = 0

correct_solution_qubo[()]=0

#correct_solution_qubo[30]=1

bqm.energy(correct_solution_qubo)


-145.0

In [59]:
from dwave.system import DWaveSampler, EmbeddingComposite
# Use a D-Wave system as the sampler
sampler = DWaveSampler() 

print("QPU {} was selected.".format(sampler.solver.name))

QPU Advantage_system1.1 was selected.


In [60]:
# Set up a D-Wave system as the sampler
sampler = EmbeddingComposite(sampler)


In [97]:
bqm_graph = bqm.to_networkx_graph()

In [100]:
print('Number of problem nodes (variables):',len(bqm_graph.nodes))

Number of problem nodes (variables): 730


In [101]:
print('Number of problem edges (couplings):',len(bqm_graph.edges))

Number of problem edges (couplings): 8520


In [64]:
from dimod import BinaryQuadraticModel
from dwave.embedding import embed_bqm, unembed_sampleset
from dwave.system.samplers import DWaveSampler
from minorminer import find_embedding
from dwave.embedding.chain_breaks import majority_vote

solver = DWaveSampler()

__, target_edgelist, target_adjacency = solver.structure

#bqm = BinaryQuadraticModel.from_qubo(Q)

emb = find_embedding(bqm.to_qubo()[0], target_edgelist)

#embedded_bqm = embed_bqm(bqm, emb, target_adjacency)

#result = solver.sample(embedded_bqm, num_reads=1)

#unembedded = unembed_sampleset(result, emb, bqm, chain_break_method=majority_vote(bqm, emb))

KeyboardInterrupt: embedding cancelled by keyboard interrupt

In [93]:
from dwave.embedding.pegasus import find_clique_embedding

find_clique_embedding(k=bqm.to_networkx_graph(), m=100)



ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-93-df2ee60d90d1>", line 3, in <module>
    find_clique_embedding(k=bqm.to_networkx_graph(), m=100)
  File "<decorator-gen-904>", line 2, in find_clique_embedding
  File "/home/julian/.local/lib/python3.9/site-packages/networkx/utils/decorators.py", line 303, in _nodes_or_number
    return func_to_be_decorated(*new_args, **kw)
  File "/home/julian/.local/lib/python3.9/site-packages/minorminer/utils/pegasus.py", line 105, in find_clique_embedding
    embedding_processor, embedding_to_pegasus = _pegasus_fragment_helper(m, target_graph)
  File "/home/julian/.local/lib/python3.9/site-packages/minorminer/utils/pegasus.py", line 60, in _pegasus_fragment_helper
    embedding_processor = processor(fragment_edges, M=m*6, N=m*6, L=2, linear=False)
  File "/home/julian/.local/lib/python3.9/

TypeError: object of type 'NoneType' has no len()

In [103]:
len(target_adjacency)

5436

In [72]:
embedded_bqm = embed_bqm(source_bqm=bqm, target_adjacency=target_adjacency)

AttributeError: 'NoneType' object has no attribute 'items'

In [66]:
embedded_bqm = embed_bqm(bqm, emb, target_adjacency)

NameError: name 'emb' is not defined

In [ ]:
result = solver.sample(embedded_bqm, num_reads=1)

In [ ]:
unembedded = unembed_sampleset(result, emb, bqm, chain_break_method=majority_vote(bqm, emb))

In [ ]:
import dwave.inspector
dwave.inspector.show(sampleset)   